<a href="https://colab.research.google.com/github/terrysu64/Machine-Learning-and-Data-Science-Projects/blob/main/introduction_to_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP Fundamentals

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-08-11 21:21:48--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-11 21:21:48 (73.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

### import dataset
kaggle: https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-08-11 21:21:52--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-08-11 21:21:52 (133 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
unzip_data('nlp_getting_started.zip')

### visualize text dataset

In [5]:
import pandas as pd
train_df = pd.read_csv('train.csv').sample(frac=1,
                                           random_state=42)
test_df = pd.read_csv('test.csv').sample(frac=1,
                                         random_state=42)
train_df.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [7]:
len(train_df), len(test_df)

(7613, 3263)

In [8]:
from random import randint
for _ in range(3):
  i = randint(1,7612)
  print(f"{'disaster' if train_df.target[i] else 'not disaster'}: {train_df.text[i]}")

disaster: .@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad
disaster: Something Frightening is Happening to the Weather in the Middle East http://t.co/bDOTQ8dSlN
not disaster: Welcome @djryanwolf @djcoreygrand @djknyce @djoneplustwo @OfficialCoreDJs #Family #Cleveland #StandUp @IAMTONYNEAL http://t.co/P6GqmCTgLj


### split data

In [9]:
from sklearn.model_selection import train_test_split
X_train, x_val, y_train, y_val = train_test_split(train_df.text.to_numpy(),
                                                  train_df.target.to_numpy(),
                                                  test_size=0.1,
                                                  random_state=42)

In [10]:
train_df.head().to_numpy() #changes df => numpy arr

array([[3796, 'destruction', nan,
        'So you have a new weapon that can cause un-imaginable destruction.',
        1],
       [3185, 'deluge', nan,
        'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@',
        0],
       [7769, 'police', 'UK',
        'DT @georgegalloway: RT @Galloway4Mayor: \x89ÛÏThe CoL police can catch a pickpocket in Liverpool Stree... http://t.co/vXIn1gOq4Q',
        1],
       [191, 'aftershock', nan,
        'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.',
        0],
       [9810, 'trauma', 'Montgomery County, MD',
        'in response to trauma Children of Addicts develop a defensive self - one that decreases vulnerability. (3',
        0]], dtype=object)

### converting text => numbers

In [11]:
#tokenization: word => a specific number
#embedding: representing relationships between tokens (can limit size + can be learned)

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

MAX_VOCAB=10000
MAX_LENGTH=15
vectorizer = TextVectorization(max_tokens=MAX_VOCAB, #limit to how many words in vocab
                               standardize="lower_and_strip_punctuation",
                               split="whitespace",
                               ngrams=None, #create grops of n words
                               output_mode="int",
                               output_sequence_length=MAX_LENGTH, #how long to limit tweet
                               pad_to_max_tokens=True #replace anything not in max_tokens to 0
)

In [12]:
vectorizer.adapt(X_train)

In [14]:
vectorizer(['hi there buddy'])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1663,   74,    1,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [15]:
vocab = vectorizer.get_vocabulary()
vocab[:5] #top 5 words

['', '[UNK]', 'the', 'a', 'in']

### creating embedding layer

In [16]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=MAX_VOCAB,
                             output_dim=128,
                             input_length=MAX_LENGTH) #neural networks work well with %8==0

embedding(vectorizer("hi there buddy"))

<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[-0.04031612,  0.00187918,  0.04536979, ..., -0.01102556,
        -0.00689974, -0.00741322],
       [-0.03306345,  0.03686079,  0.01701177, ...,  0.00701331,
         0.0412286 , -0.04609409],
       [-0.04369364,  0.04763517, -0.01739151, ...,  0.03852138,
         0.01071489, -0.01067911],
       ...,
       [-0.03790686,  0.04212775, -0.02348931, ..., -0.00018896,
        -0.01221972, -0.02255811],
       [-0.03790686,  0.04212775, -0.02348931, ..., -0.00018896,
        -0.01221972, -0.02255811],
       [-0.03790686,  0.04212775, -0.02348931, ..., -0.00018896,
        -0.01221972, -0.02255811]], dtype=float32)>

## building models for the dataset